In [1]:
from datasets import load_dataset

dataset = load_dataset("duxprajapati/symptom-disease-dataset")

C:\Users\GamerPc\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5634
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1409
    })
})

In [3]:
#show the number of Name column which is not duplicated
#show that unique element of dataset["train"]['Name'] which is list type
labels= dataset["train"]['label']

unique_names = set(labels)


In [4]:
len(unique_names)

866

In [5]:
labels= dataset["test"]['label']

In [6]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
from transformers import AutoModel
import torch

# Load the base model
model_base = AutoModelForSequenceClassification.from_pretrained("duxprajapati/symptom-disease-model")


config = AutoConfig.from_pretrained("duxprajapati/symptom-disease-model")

In [7]:
print(model_base)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
import nltk
from nltk.corpus import wordnet
from random import randint

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def synonym_replacement(sentence, num_replacements=1):
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    replacements = 0
    for i, (word, tag) in enumerate(pos_tags):
        if replacements >= num_replacements or wordnet.synsets(word) == []:
            continue
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = synonyms[randint(0, len(synonyms) - 1)]
            words[i] = synonym
            replacements += 1

    return ' '.join(words)

# Example usage
sentence = "Fever and cough might indicate a flu infection."
augmented_sentence = synonym_replacement(sentence, 2)
print("Original:", sentence)
print("Augmented:", augmented_sentence)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\GamerPc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GamerPc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original: Fever and cough might indicate a flu infection.
Augmented: febricity and coughing might indicate a flu infection .


In [9]:
from datasets import load_dataset

# Assuming the functions `get_synonyms` and `synonym_replacement` are defined as above

# Load your dataset
dataset = load_dataset("duxprajapati/symptom-disease-dataset")

# Function to augment the data
def augment_dataset(data, augmentation_function, num_replacements=1):
    augmented_texts = []
    labels = []
    for example in data:
        augmented_text = augmentation_function(example['text'], num_replacements)
        augmented_texts.append(augmented_text)
        labels.append(example['label'])
    return augmented_texts, labels

# Augment the training data
augmented_texts, labels = augment_dataset(dataset['train'], synonym_replacement, 2)

# Create a new dataset from the augmented texts and labels
augmented_dataset = {
    'text': augmented_texts + [example['text'] for example in dataset['train']],
    'label': labels + [example['label'] for example in dataset['train']]
}


In [10]:
augmented_dataset

{'text': ["single cause been having migraines and headaches . I ca n't sleep . My whole body is shaking and shivering . I feel dizzy sometimes .",
  'iodin own asthma and I get wheezing and breathing problems . I also have fevers , headaches , and I feel tired all the time .',
  "contract and symptom of primary ovarian insufficiency are similar to those of menopause or estrogen deficiency . They include : Irregular or skipped periods , which might be present for years or develop after a pregnancy or after stopping birth control pills Difficulty getting pregnant Hot flashes Night sweats Vaginal dryness Dry eyes Irritability or difficulty concentrating Decreased sexual desire When to see a health care provider If you 've missed your period for three months or more , see your health care provider to determine the cause . You can miss your period for a number of reasons — including pregnancy , stress , or a change in diet or exercise habits — but it 's best to get evaluated whenever your m

In [11]:
# Assuming 'dataset' is your original DatasetDict containing 'train' and 'test' splits
# And assuming 'augmented_texts' and 'labels' are your lists of augmented data and their corresponding labels

# Convert augmented data into a list of dictionaries
augmented_data = [{'text': text, 'label': label} for text, label in zip(augmented_texts, labels)]

# Append augmented data to the original dataset
for item in augmented_data:
    dataset['train'] = dataset['train'].add_item(item)

# Now, 'dataset['train']' contains both the original and augmented data as separate rows


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11268
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1409
    })
})

In [13]:
# from googletrans import Translator, LANGUAGES

# translator = Translator()

# def translate_augment(text, lang='es'):
#     # Translate to the target language
#     translated = translator.translate(text, dest=lang).text
#     # Translate back to the original language
#     retranslated = translator.translate(translated, dest='en').text
#     return retranslated


In [14]:
# # Function to augment the data
# def augment_dataset_2(data, augmentation_function):
#     augmented_texts = []
#     labels = []
#     for example in data:
#         augmented_text = augmentation_function(example['text'],)
#         augmented_texts.append(augmented_text)
#         labels.append(example['label'])
#     return augmented_texts, labels

# # Augment the training data
# augmented_texts, labels = augment_dataset_2(dataset['train'], translate_augment)

# # Create a new dataset from the augmented texts and labels
# augmented_dataset = {
#     'text': augmented_texts + [example['text'] for example in dataset['train']],
#     'label': labels + [example['label'] for example in dataset['train']]
# }


In [15]:
# Assuming 'dataset' is your original DatasetDict containing 'train' and 'test' splits
# And assuming 'augmented_texts' and 'labels' are your lists of augmented data and their corresponding labels

# Convert augmented data into a list of dictionaries
# augmented_data = [{'text': text, 'label': label} for text, label in zip(augmented_texts, labels)]

# # Append augmented data to the original dataset
# for item in augmented_data:
#     dataset['train'] = dataset['train'].add_item(item)

# Now, 'dataset['train']' contains both the original and augmented data as separate rows

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11268
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1409
    })
})

In [17]:
#fine tune the model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=24,
    num_train_epochs=3,
    learning_rate=2e-8,
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    output_dir='./results',
    overwrite_output_dir=True,
)





In [18]:
#tokenize the data
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("duxprajapati/symptom-disease-model")

tokenized_dataset = dataset.map(lambda examples: tokenizer(examples['text'], padding='max_length', truncation=True), batched=True)


Map: 100%|██████████| 11268/11268 [00:02<00:00, 5543.69 examples/s]


In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 11268
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1409
    })
})

In [20]:
trainer = Trainer(
    model=model_base,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

c:\Users\GamerPc\.conda\envs\tensorflow\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
#show the result of the model
trainer.train()


  1%|          | 10/1410 [02:07<4:55:23, 12.66s/it]

{'loss': 0.343, 'grad_norm': 4.519108772277832, 'learning_rate': 1.9858156028368796e-08, 'epoch': 0.02}


  1%|▏         | 20/1410 [04:15<4:56:38, 12.80s/it]

{'loss': 0.239, 'grad_norm': 1.4740196466445923, 'learning_rate': 1.9716312056737588e-08, 'epoch': 0.04}


  2%|▏         | 30/1410 [06:23<4:54:11, 12.79s/it]

...

 99%|█████████▉| 1400/1410 [5:02:00<02:09, 12.96s/it]

{'loss': 0.2406, 'grad_norm': 2.1115171909332275, 'learning_rate': 1.4184397163120568e-10, 'epoch': 2.98}


100%|██████████| 1410/1410 [5:04:03<00:00, 12.94s/it]

{'loss': 0.2684, 'grad_norm': 7.029175758361816, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 18243.5609, 'train_samples_per_second': 1.853, 'train_steps_per_second': 0.077, 'train_loss': 0.28651398555606816, 'epoch': 3.0}


TrainOutput(global_step=1410, training_loss=0.28651398555606816, metrics={'train_runtime': 18243.5609, 'train_samples_per_second': 1.853, 'train_steps_per_second': 0.077, 'train_loss': 0.28651398555606816, 'epoch': 3.0})